In [140]:
import pandas as pd
import datetime
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error as MSE
from xgboost import XGBRegressor

In [6]:
dfL1Data = pd.read_csv(r'./ABC_Tick_Data/ABC_Level_One_Tick_Data.csv')

In [90]:
# convert Time_Hour to datetime objects and seperate date from time
dfL1Data['Time_Hour_Formatted'] = dfL1Data.Time_Hour.astype('datetime64')

dfL1Data['Date'] = dfL1Data.apply(lambda x: x['Time_Hour_Formatted'].date(),axis=1)
dfL1Data['Time'] = dfL1Data.apply(lambda x: x['Time_Hour_Formatted'].time(),axis=1)

# remove trade after 5pm
dfL1Transformed = dfL1Data.copy()
dfL1Transformed = dfL1Transformed[dfL1Transformed['Time']<datetime.time(17,0,0)]

# keep useful features only
useful_features = ['Date','Time','VWAP','Volume','Avg_Bid_Ask_Spread']
dfSelectedFeatures = dfL1Transformed[useful_features]


dfSelectedFeatures

,Date,Time,VWAP,Volume,Avg_Bid_Ask_Spread
0,2018-01-04,08:00:00,53.951161,2.546155e+05,-0.014760
1,2018-01-04,09:00:00,54.224500,7.781718e+04,0.013481
2,2018-01-04,10:00:00,54.265969,5.819173e+04,0.011557
3,2018-01-04,11:00:00,54.183729,3.476209e+04,0.011537
4,2018-01-04,12:00:00,54.158934,3.464418e+04,-0.014078
...,...,...,...,...,...
1018,2018-06-14,11:00:00,34.816504,1.527895e+05,0.004185
1019,2018-06-14,12:00:00,34.581861,1.829505e+05,0.007030
1020,2018-06-14,13:00:00,34.539470,2.995654e+05,0.008004
1021,2018-06-14,14:00:00,34.445519,3.029528e+05,0.007508


In [109]:
# aggregate data by date with weighted sum of spreads

dfRollingVolume_ = dfSelectedFeatures.groupby('Date',as_index=False)['Volume'].sum()
dfRollingVolume_ = pd.merge(left=dfSelectedFeatures,right=dfRollingVolume_,on='Date',suffixes=[None,'_Rolling'],how='left')
dfRollingVolume_['Weight'] = dfRollingVolume_['Volume']/dfRollingVolume_['Volume_Rolling']
dfRollingVolume_['Weighted_Spread'] = dfRollingVolume_['Weight'] * dfRollingVolume_['Avg_Bid_Ask_Spread']
dfWeighted_ = dfRollingVolume_.groupby('Date',as_index=False)[['VWAP','Volume','Weighted_Spread']] \
.aggregate({'VWAP':'mean','Volume':sum,'Weighted_Spread':sum})

# assuming an asset needs to be sold in less than 30 days
windows = [1,2,3,4,5,10,15,30]
lookback_range = 7

df_lst = []

for w in windows:
    df = dfWeighted_.rolling(w).aggregate({'VWAP':'mean','Volume':sum,'Weighted_Spread':sum})
    df['Window'] = w
    
    dfJoin = df.copy()
    
    # use previous 7 days data as regressors
    for i in range(1,lookback_range):
        dfShift = df.shift(i)
        dfJoin = dfJoin.join(dfShift.drop(columns='Window'),rsuffix='_'+str(i))
    df_lst.append(dfJoin)
    
dfFeatures = pd.concat(df_lst, ignore_index=True)

dfFeatures

,VWAP,Volume,Weighted_Spread,Window,VWAP_1,Volume_1,Weighted_Spread_1,VWAP_2,Volume_2,Weighted_Spread_2,...,Weighted_Spread_3,VWAP_4,Volume_4,Weighted_Spread_4,VWAP_5,Volume_5,Weighted_Spread_5,VWAP_6,Volume_6,Weighted_Spread_6
0,54.120559,1.495614e+06,-0.083702,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53.838618,1.348446e+06,-0.138434,1,54.120559,1.495614e+06,-0.083702,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53.087938,1.479861e+06,-0.186802,1,53.838618,1.348446e+06,-0.138434,54.120559,1.495614e+06,-0.083702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53.126425,1.278397e+06,-0.220335,1,53.087938,1.479861e+06,-0.186802,53.838618,1.348446e+06,-0.138434,...,-0.083702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52.696743,1.641908e+06,-0.101092,1,53.126425,1.278397e+06,-0.220335,53.087938,1.479861e+06,-0.186802,...,-0.138434,54.120559,1.495614e+06,-0.083702,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,36.483664,8.979301e+07,-2.123530,30,36.452140,8.704672e+07,-2.130776,36.467322,8.623961e+07,-2.156703,...,-2.175385,36.616126,8.450332e+07,-2.254030,36.704137,8.501591e+07,-2.279091,36.818660,8.480932e+07,-2.140608
892,36.524798,9.098291e+07,-1.800772,30,36.483664,8.979301e+07,-2.123530,36.452140,8.704672e+07,-2.130776,...,-2.156703,36.526509,8.396247e+07,-2.175385,36.616126,8.450332e+07,-2.254030,36.704137,8.501591e+07,-2.279091
893,36.537596,9.395690e+07,-1.749003,30,36.524798,9.098291e+07,-1.800772,36.483664,8.979301e+07,-2.123530,...,-2.130776,36.467322,8.623961e+07,-2.156703,36.526509,8.396247e+07,-2.175385,36.616126,8.450332e+07,-2.254030
894,36.506149,9.467366e+07,-1.661449,30,36.537596,9.395690e+07,-1.749003,36.524798,9.098291e+07,-1.800772,...,-2.123530,36.452140,8.704672e+07,-2.130776,36.467322,8.623961e+07,-2.156703,36.526509,8.396247e+07,-2.175385


In [116]:
# log transform all Volume features

volume_features = ['Volume']
volume_features.extend(["Volume_"+str(i) for i in range (1,lookback_range)])
dfFeaturesX = dfFeatures.copy()

for c in volume_features:
    dfFeaturesX[c] = np.log(dfFeatures[c])

dfFeaturesX

,VWAP,Volume,Weighted_Spread,Window,VWAP_1,Volume_1,Weighted_Spread_1,VWAP_2,Volume_2,Weighted_Spread_2,...,Weighted_Spread_3,VWAP_4,Volume_4,Weighted_Spread_4,VWAP_5,Volume_5,Weighted_Spread_5,VWAP_6,Volume_6,Weighted_Spread_6
0,54.120559,14.218047,-0.083702,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,53.838618,14.114463,-0.138434,1,54.120559,14.218047,-0.083702,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,53.087938,14.207459,-0.186802,1,53.838618,14.114463,-0.138434,54.120559,14.218047,-0.083702,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53.126425,14.061117,-0.220335,1,53.087938,14.207459,-0.186802,53.838618,14.114463,-0.138434,...,-0.083702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,52.696743,14.311370,-0.101092,1,53.126425,14.061117,-0.220335,53.087938,14.207459,-0.186802,...,-0.138434,54.120559,14.218047,-0.083702,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
891,36.483664,18.313018,-2.123530,30,36.452140,18.281956,-2.130776,36.467322,18.272640,-2.156703,...,-2.175385,36.616126,18.252301,-2.254030,36.704137,18.258349,-2.279091,36.818660,18.255916,-2.140608
892,36.524798,18.326182,-1.800772,30,36.483664,18.313018,-2.123530,36.452140,18.281956,-2.130776,...,-2.156703,36.526509,18.245880,-2.175385,36.616126,18.252301,-2.254030,36.704137,18.258349,-2.279091
893,36.537596,18.358347,-1.749003,30,36.524798,18.326182,-1.800772,36.483664,18.313018,-2.123530,...,-2.130776,36.467322,18.272640,-2.156703,36.526509,18.245880,-2.175385,36.616126,18.252301,-2.254030
894,36.506149,18.365946,-1.661449,30,36.537596,18.358347,-1.749003,36.524798,18.326182,-1.800772,...,-2.123530,36.452140,18.281956,-2.130776,36.467322,18.272640,-2.156703,36.526509,18.245880,-2.175385


In [21]:
dfFeaturesSelected

,Volume,Avg_Bid_Ask_Spread,TWAP_1,Volume_1,Avg_Bid_Ask_Spread_1,TWAP_2,Volume_2,Avg_Bid_Ask_Spread_2,TWAP_3,Volume_3,...,TWAP_4,Volume_4,Avg_Bid_Ask_Spread_4,TWAP_5,Volume_5,Avg_Bid_Ask_Spread_5,TWAP_6,Volume_6,Avg_Bid_Ask_Spread_6,window
0,1.495614e+06,-0.027190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,1.348446e+06,-0.047058,54.156305,1.495614e+06,-0.027190,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1.479861e+06,-0.045502,53.735362,1.348446e+06,-0.047058,54.156305,1.495614e+06,-0.027190,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,1.278397e+06,-0.051341,53.093352,1.479861e+06,-0.045502,53.735362,1.348446e+06,-0.047058,54.156305,1.495614e+06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,1.641908e+06,-0.028079,53.129402,1.278397e+06,-0.051341,53.093352,1.479861e+06,-0.045502,53.735362,1.348446e+06,...,54.156305,1.495614e+06,-0.027190,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,2.993100e+06,-0.007405,36.450699,2.901557e+06,-0.007309,36.466447,2.874654e+06,-0.007611,36.523039,2.798749e+06,...,36.610358,2.816777e+06,-0.008717,36.698975,2.833864e+06,-0.008830,36.810710,2.826977e+06,-0.008315,30
780,3.032764e+06,-0.004365,36.482372,2.993100e+06,-0.007405,36.450699,2.901557e+06,-0.007309,36.466447,2.874654e+06,...,36.523039,2.798749e+06,-0.007881,36.610358,2.816777e+06,-0.008717,36.698975,2.833864e+06,-0.008830,30
781,3.131897e+06,-0.005690,36.523928,3.032764e+06,-0.004365,36.482372,2.993100e+06,-0.007405,36.450699,2.901557e+06,...,36.466447,2.874654e+06,-0.007611,36.523039,2.798749e+06,-0.007881,36.610358,2.816777e+06,-0.008717,30
782,3.155789e+06,-0.005177,36.526004,3.131897e+06,-0.005690,36.523928,3.032764e+06,-0.004365,36.482372,2.993100e+06,...,36.450699,2.901557e+06,-0.007309,36.466447,2.874654e+06,-0.007611,36.523039,2.798749e+06,-0.007881,30


# Modelling

In [154]:
X_train, X_test, y_train, y_test = train_test_split(dfFeaturesX.loc[:,dfFeaturesX.columns != 'Weighter_Spread'].to_numpy(), dfFeaturesX['Weighted_Spread'].to_numpy(), test_size=0.33, random_state=42)

In [161]:
xgbr = XGBRegressor(objective ='reg:squarederror',n_estimators = 200, seed = 12)

In [162]:
xgbr.fit(X_train, y_train)

XGBRegressor(n_estimators=200, objective='reg:squarederror', seed=12)

In [163]:
pred = xgbr.predict(X_test)
pred

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
       0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.

In [164]:
rmse = np.sqrt(MSE(y_test, pred))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').